<a href="https://colab.research.google.com/github/CodeHunterOfficial/ABC_DataMining/blob/main/TimeSeries/TimeSeries-2025/%D0%9B%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%BE%D1%80%D0%BD%D1%8B%D0%B9_%D0%BF%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D1%83%D0%BC_%E2%84%96_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# 🧪 **Лабораторный практикум № 3**  
## **Сравнительный анализ классических моделей временных рядов**

**Кафедра:** Кафедра анализа данных и технологий программирования  
**Дисциплина:** Машинное обучение в задачах прогнозирования  
**Уровень:** Магистратура, 2 курс  
**Преподаватель:** Арабов Муллошараф Курбонович  
**Консультации:**   

---

## 🎯 1. Цели и задачи работы

### **Цель:**  
Освоить полный цикл **сравнительного анализа классических моделей временных рядов**, включая базовые (AR, ARIMA), модели волатильности (GARCH), сложные сезонные (TBATS, Prophet) и многомерные (VAR, VECM), с использованием **единообразной методологии**, **статистической диагностики** и **обоснованного выбора** на основе метрик и тестов значимости.

### **Задачи:**
1. Привести ряд к стационарности с помощью **лог-трансформации**, **преобразования Бокса–Кокса** и **дифференцирования** (обычного, сезонного, комбинированного).
2. Проверить стационарность с помощью **ADF/KPSS** после каждого шага и выбрать оптимальную цепочку преобразований.
3. Выполнить **расширенный feature engineering**: лаги (1, 2, 7, 30), скользящие статистики (mean, std, min, max по окнам 7, 30), временные признаки (день недели, месяц, квартал, праздники), экзогенные переменные.
4. Разделить данные на **обучающую (70%)**, **валидационную (15%)** и **тестовую (15%)** выборки в хронологическом порядке.
5. Реализовать и сравнить **стратегии прогнозирования**:
   - Рекурсивную,
   - Прямую,
   - Гибридную,
   - для **трёх горизонтов**: **h = 1, 7, 30**.
6. Провести **кросс-валидацию** с расширяющимся и скользящим окном (`TimeSeriesSplit`).
7. Построить следующие модели:
   - **Базовые**: AR, MA, ARMA, ARIMA, SARIMA, SARIMAX  
   - **Волатильность**: GARCH (на остатках ARIMA)  
   - **Сложные/сезонные**: TBATS, Prophet  
   - **Многомерные**: VAR, VECM (при наличии ≥2 рядов)  
   - **Бенчмарки**: Naive, Seasonal Naive, Simple Exponential Smoothing (SES)
8. Провести **диагностику адекватности**:
   - Тест Льюнга–Бокса на автокорреляцию остатков,
   - График остатков vs прогнозов (гомоскедастичность),
   - Q-Q plot и тест Шапиро–Уилка (нормальность),
   - ACF/PACF остатков.
9. Оценить качество по метрикам: **MAE, RMSE, MAPE, MASE, SMAPE, R², RMSLE**.
10. Применить **тест Diebold–Mariano** для статистической проверки значимости различий между моделями.
11. Составить **сравнительную таблицу** с параметрами, метриками, временем выполнения и ранжированием.
12. Разработать **веб-интерфейс** для интерактивного сравнения моделей.
13. Сформулировать **практические рекомендации по использованию моделей в продакшене**.

> 💡 **Важно:** работа фокусируется **исключительно на классических методах** — нейросети и градиентный бустинг **не используются**.

> 📌 **Требования к объёму данных:**  
> - Обучающая выборка: **не менее 500 наблюдений**  
> - Тестовая выборка: **не менее 2 × max(h)** и **минимум 50 точек**  
> - Горизонты прогноза: **h ∈ {1, 7, 30}**

---

## 📚 2. Теоретические предпосылки

Каждая модель временного ряда основана на определённых **статистических предположениях**:

- **AR/MA/ARMA** требуют стационарности.
- **ARIMA/SARIMA** добавляют дифференцирование для устранения тренда/сезонности.
- **SARIMAX** позволяет учитывать внешние факторы (праздники, промо и др.).
- **GARCH** моделирует **динамику волатильности**, а не уровень ряда.
- **TBATS** автоматически обрабатывает **множественную и нестационарную сезонность**.
- **Prophet** устойчив к пропускам и выбросам, но менее гибок в настройке сезонности.
- **VAR/VECM** применяются только к **многомерным** рядам; VECM требует коинтеграции.
- **Бенчмарки** (Naive, Seasonal Naive, SES) служат нижней границей качества.

> 🔑 **Ключевые инструменты**:
> - Преобразование Бокса–Кокса (`scipy.stats.boxcox`, `boxcox_normmax`) — стабилизация дисперсии.
> - `pmdarima.auto_arima` — автоматический подбор SARIMA.
> - `statsmodels`, `arch`, `tbats`, `prophet` — реализация моделей.
> - `sklearn.model_selection.TimeSeriesSplit` — корректная кросс-валидация.
> - Тест **Diebold–Mariano** (`statsmodels.stats.diagnostic.dm_test`) — сравнение прогнозов.

---

## 🧪 3. Методика эксперимента и порядок выполнения

### **3.1. Этап 1. Предобработка и преобразования**

- Применить **лог-трансформацию** (если y > 0).
- Выполнить **Бокса–Кокса**:  
  ```python
  λ = boxcox_normmax(y_train, brack=(-2, 2))
  y_bc, _ = boxcox(y_train, lmbda=λ)
  ```
- Дифференцирование: `diff(1)`, `diff(s)`, `diff(1).diff(s)`.
- Проверить стационарность: **ADF (H₀: нестационарность)**, **KPSS (H₀: стационарность)**.
- Выбрать цепочку, минимизирующую противоречия между тестами.

---

### **3.2. Этап 2. Разбиение данных и feature engineering**

- **Хронологическое разбиение**:  
  - train: 70%  
  - val: 15% (для настройки гиперпараметров)  
  - test: 15% (финальная оценка)
- **Признаки**:
  - Лаги: 1, 2, 7, 30
  - Скользящие: mean/std/min/max (7, 30)
  - Временные: `dayofweek`, `month`, `is_holiday`
  - Циклические: `sin(2π·t/7)`, `cos(2π·t/12)`
  - Экзогенные: для SARIMAX и Prophet

---

### **3.3. Этап 3. Стратегии прогнозирования и горизонты**

- Протестировать все модели на **h = 1, 7, 30**.
- Для моделей без нативного многопшагового прогноза:
  - **Рекурсивная**: `ŷ_{t+1} → вход для ŷ_{t+2}`  
  - **Прямая**: отдельная модель для каждого шага  
  - **Гибридная**: рекурсивная для h≤3, прямая — для h>3
- Оценить **накопление ошибки** и **вычислительную сложность**.

---

### **3.4. Этап 4. Кросс-валидация**

- Реализовать:
  - **Расширяющееся окно**: [0:t] → [t+1:t+h]
  - **Скользящее окно**: [t−w:t] → [t+1:t+h]
  - `TimeSeriesSplit(n_splits=5, max_train_size=365)`
- Усреднить метрики по фолдам.
- Визуализировать динамику ошибки во времени.

---

### **3.5. Этап 5. Построение моделей**

| Группа | Модели | Особенности |
|-------|--------|-------------|
| **Базовые** | AR, MA, ARMA, ARIMA, SARIMA, SARIMAX | Подбор через AIC/BIC или `auto_arima` |
| **Волатильность** | GARCH(1,1) | Обучается на остатках ARIMA |
| **Сезонные** | TBATS, Prophet | TBATS — для сложной сезонности; Prophet — для устойчивости |
| **Многомерные** | VAR, VECM | Требуют ≥2 рядов; VECM — только при коинтеграции |
| **Бенчмарки** | Naive, Seasonal Naive, SES | SES как часть `ExponentialSmoothing` |

> Для всех моделей: прогноз, доверительные интервалы (если поддерживается), обратное преобразование Бокса–Кокса.

---

### **3.6. Этап 6. Диагностика моделей**

Для каждой модели и каждого горизонта:
- **Тест Льюнга–Бокса** (p > 0.05 → остатки — белый шум)
- **Гомоскедастичность**: визуальный анализ + (опционально) Breusch–Pagan
- **Нормальность**: Q-Q plot + **тест Шапиро–Уилка**
- **ACF/PACF остатков**
- **Визуализация**: ряд, прогноз, остатки, CI

> Диагностика обязательна для **топ-3 моделей** по метрикам.

---

### **3.7. Этап 7. Оценка качества и статистическое сравнение**

- **Метрики**: MAE, RMSE, MAPE, MASE, SMAPE, R², **RMSLE** (для лог-рядов)
- **Тест Diebold–Mariano**:
  ```python
  from statsmodels.stats.diagnostic import dm_test
  dm_test(forecast1, forecast2, test='two_sided')
  ```
- **Сравнительная таблица**:
  - Параметры модели
  - Все метрики на test для h=1,7,30
  - p-value (Ljung–Box, Shapiro–Wilk, Diebold–Mariano)
  - Время обучения
  - Статус стационарности
- **Ранжирование** по взвешенной оценке (например, по MASE и p(LB)).

---

### **3.8. Этап 8. Веб-интерфейс**

**Обязательные элементы**:
- Загрузка CSV/Parquet
- Выбор целевой переменной и горизонта (1/7/30)
- Переключение моделей (включая бенчмарки)
- Настройка преобразований (авто/ручной λ)
- Визуализация: прогноз + CI + остатки
- Таблица метрик и ранжирования
- Экспорт прогноза

**Стек**: `Streamlit` + `Plotly` + `statsmodels` + `prophet` + `pmdarima`

> Деплой на **Hugging Face Spaces** или **Render**.

---

### **3.9. Этап 9. Выводы и рекомендации**

- **Сводная таблица** с ранжированием по каждому горизонту.
- **Обоснование выбора лучшей модели** с учётом:
  - Качества (метрики),
  - Адекватности (тесты),
  - Интерпретируемости,
  - Вычислительной сложности.
- **Рекомендации по продакшену**:
  - Когда использовать Prophet vs SARIMAX?
  - Нужна ли TBATS при простой сезонности?
  - Как обновлять модель при поступлении новых данных?
- **Рефлексия**: сложности, инсайты, ограничения.

---

## 🔍 4. Дополнительные задания (по желанию)

1. Анализ чувствительности к λ в Боксе–Коксе  
2. Эмпирическое покрытие прогнозных интервалов  
3. Ансамбль из топ-3 моделей  
4. Автоматическое обнаружение структурных сдвигов  
5. Байесовская оптимизация гиперпараметров SARIMAX

---

## 📄 5. Требования к отчёту

1. Титульный лист  
2. Описание данных  
3. Цепочка преобразований + ADF/KPSS  
4. Примеры feature engineering  
5. Сравнение стратегий на h=1,7,30  
6. Результаты кросс-валидации  
7. Сравнительная таблица **всех моделей**  
8. Диагностика **топ-3** (Ljung–Box, остатки, Q-Q)  
9. Результаты **Diebold–Mariano**  
10. Скриншоты/ссылка на веб-интерфейс  
11. Выводы и рекомендации по продакшену

---

## 📊 6. Критерии оценки

| Оценка | Критерии |
|--------|---------|
| **5** | Все модели реализованы единообразно. Есть 70/15/15 разбиение, три горизонта, Diebold–Mariano, диагностика, веб-интерфейс. Результаты воспроизводимы. |
| **4** | Реализованы основные модели, есть метрики и диагностика, но без Diebold–Mariano или одного из бенчмарков. |
| **3** | Построены 5–7 моделей, есть сравнение, но без полной диагностики или стратегий. |
| **2** | Работа не выполнена или отсутствует сравнительный анализ. |

---

## 📖 7. Литература

1. Hyndman, R.J., Athanasopoulos, G. *Forecasting: Principles and Practice*, 3rd ed. — [https://otexts.com/fpp3/](https://otexts.com/fpp3/)  
2. Box, G.E.P., Jenkins, G.M. *Time Series Analysis: Forecasting and Control*  
3. Документация:  
   - `scipy.stats.boxcox` — [SciPy](https://docs.scipy.org/doc/scipy/reference/stats.html)  
   - `statsmodels` — [Statsmodels](https://www.statsmodels.org)  
   - `pmdarima`, `prophet`, `tbats`  
   - `sklearn.TimeSeriesSplit`  
   - `Streamlit` — [https://docs.streamlit.io](https://docs.streamlit.io)  
   - `Plotly` — [https://plotly.com/python/](https://plotly.com/python/)

---

> 💡 Эта работа завершает цикл по **классическим методам** и создаёт основу для перехода к **современным ML/нейросетевым подходам** в последующих лабораторных.

